# VizFlow Demo - v0.4.1

Demonstrates **column mapping presets** and **schema evolution** with real data.

In [1]:
import polars as pl
import vizflow as vf
from pathlib import Path

## 1. Trade Data (ylin preset)

Load meords trade files with `column_preset="ylin"` - auto-renames 52 columns.

In [2]:
config = vf.Config(
    trade_dir=Path("data/ylin/trade"),
    trade_pattern="{date}.meords",
    column_preset="ylin",
    market="CN",
    trade_schema={
        "order_qty": vf.ColumnSchema(cast_to=pl.Int64),
    },
)
vf.set_config(config)

df_trade = vf.scan_trade("11110101")
print("=== Trade Data (ylin preset) ===")
print(df_trade.select(['ukey', 'order_side', 'order_qty', 'bid_px0', 'ask_px0']).head(5).collect())

=== Trade Data (ylin preset) ===
shape: (5, 5)
┌──────────┬────────────┬───────────┬─────────┬─────────┐
│ ukey     ┆ order_side ┆ order_qty ┆ bid_px0 ┆ ask_px0 │
│ ---      ┆ ---        ┆ ---       ┆ ---     ┆ ---     │
│ i64      ┆ str        ┆ i64       ┆ f64     ┆ f64     │
╞══════════╪════════════╪═══════════╪═════════╪═════════╡
│ 11000408 ┆ Sell       ┆ 2900      ┆ 22.48   ┆ 22.5    │
│ 11002405 ┆ Buy        ┆ 27100     ┆ 7.19    ┆ 7.2     │
│ 11000739 ┆ Sell       ┆ 400       ┆ 14.97   ┆ 14.98   │
│ 11002709 ┆ Sell       ┆ 600       ┆ 14.7    ┆ 14.71   │
│ 11300436 ┆ Sell       ┆ 14000     ┆ 20.3    ┆ 20.31   │
└──────────┴────────────┴───────────┴─────────┴─────────┘


## 2. Alpha Data (jyao_v20251114 preset)

Load alpha files with `column_preset="jyao_v20251114"` - auto-renames alpha columns.

In [3]:
config = vf.Config(
    alpha_dir=Path("data/jyao/alpha"),
    alpha_pattern="alpha_{date}.feather",
    column_preset="jyao_v20251114",
    market="CN",
)
vf.set_config(config)

df_alpha = vf.scan_alpha("20251114")
print("=== Alpha Data (jyao_v20251114 preset) ===")
print(df_alpha.select(['ukey', 'ticktime', 'bid_px0', 'ask_px0', 'alpha_10s', 'alpha_60s', 'alpha_3m', 'alpha_30m']).collect())

=== Alpha Data (jyao_v20251114 preset) ===
shape: (6, 8)
┌──────────┬──────────┬─────────┬─────────┬───────────┬───────────┬──────────┬───────────┐
│ ukey     ┆ ticktime ┆ bid_px0 ┆ ask_px0 ┆ alpha_10s ┆ alpha_60s ┆ alpha_3m ┆ alpha_30m │
│ ---      ┆ ---      ┆ ---     ┆ ---     ┆ ---       ┆ ---       ┆ ---      ┆ ---       │
│ i64      ┆ i64      ┆ f64     ┆ f64     ┆ f64       ┆ f64       ┆ f64      ┆ f64       │
╞══════════╪══════════╪═════════╪═════════╪═══════════╪═══════════╪══════════╪═══════════╡
│ 11000408 ┆ 93000000 ┆ 22.48   ┆ 22.5    ┆ 0.0012    ┆ 0.0018    ┆ 0.0025   ┆ 0.005     │
│ 11000408 ┆ 93001000 ┆ 22.49   ┆ 22.51   ┆ 0.0015    ┆ 0.002     ┆ 0.0028   ┆ 0.0055    │
│ 11002405 ┆ 93000000 ┆ 7.19    ┆ 7.2     ┆ -0.0008   ┆ -0.001    ┆ -0.0015  ┆ -0.003    │
│ 11002405 ┆ 93001000 ┆ 7.2     ┆ 7.21    ┆ -0.0005   ┆ -0.0008   ┆ -0.0012  ┆ -0.0025   │
│ 11000739 ┆ 93000000 ┆ 14.97   ┆ 14.98   ┆ 0.002     ┆ 0.0028    ┆ 0.0035   ┆ 0.007     │
│ 11000739 ┆ 93001000 ┆ 14.98   ┆

## 3. Parse Timestamps

In [4]:
df_alpha = vf.parse_time(df_alpha, timestamp_col="ticktime")
print("=== Parse Timestamps ===")
print(df_alpha.select(['ukey', 'ticktime', 'tod_ticktime', 'elapsed_ticktime']).head(6).collect())

=== Parse Timestamps ===
shape: (6, 4)
┌──────────┬──────────┬──────────────┬──────────────────┐
│ ukey     ┆ ticktime ┆ tod_ticktime ┆ elapsed_ticktime │
│ ---      ┆ ---      ┆ ---          ┆ ---              │
│ i64      ┆ i64      ┆ time         ┆ i64              │
╞══════════╪══════════╪══════════════╪══════════════════╡
│ 11000408 ┆ 93000000 ┆ 09:30:00     ┆ 0                │
│ 11000408 ┆ 93001000 ┆ 09:30:01     ┆ 1000             │
│ 11002405 ┆ 93000000 ┆ 09:30:00     ┆ 0                │
│ 11002405 ┆ 93001000 ┆ 09:30:01     ┆ 1000             │
│ 11000739 ┆ 93000000 ┆ 09:30:00     ┆ 0                │
│ 11000739 ┆ 93001000 ┆ 09:30:01     ┆ 1000             │
└──────────┴──────────┴──────────────┴──────────────────┘


## 4. Aggregation

In [5]:
metrics = {
    "count": pl.len(),
    "avg_alpha_10s": pl.col("alpha_10s").mean(),
    "avg_alpha_3m": pl.col("alpha_3m").mean(),
}
agg_df = vf.aggregate(df_alpha, group_by=["ukey"], metrics=metrics)
print("=== Aggregation ===")
print(agg_df.collect())

=== Aggregation ===
shape: (3, 4)
┌──────────┬───────┬───────────────┬──────────────┐
│ ukey     ┆ count ┆ avg_alpha_10s ┆ avg_alpha_3m │
│ ---      ┆ ---   ┆ ---           ┆ ---          │
│ i64      ┆ u32   ┆ f64           ┆ f64          │
╞══════════╪═══════╪═══════════════╪══════════════╡
│ 11002405 ┆ 2     ┆ -0.00065      ┆ -0.00135     │
│ 11000739 ┆ 2     ┆ 0.0019        ┆ 0.00335      │
│ 11000408 ┆ 2     ┆ 0.00135       ┆ 0.00265      │
└──────────┴───────┴───────────────┴──────────────┘
